In [1]:
import os 
import sys 
import json 
import logging 
from pathlib import Path 
from functools import cache
from itertools import product

# Required when developing in a jupyter-notebook environment 
cur_path = os.path.abspath("../..")
if cur_path not in sys.path: 
    sys.path.append(cur_path)

import numpy as np 
import pandas as pd 
import altair as alt 
from altair import datum
from palettable.scientific.sequential import Batlow_5
from dotenv import load_dotenv
from subgrounds.subgrounds import Subgrounds, Subgraph
from subgrounds.subgraph import SyntheticField
from subgrounds.pagination import ShallowStrategy

# Required when developing in a jupyter-notebook environment 
load_dotenv('../../../../.env')

from utils_notebook.utils import ddf, remove_prefix, load_subgraph, remove_keys
from utils_notebook.vega import (
    output_chart, 
    apply_css, 
    wide_to_longwide, 
    chart, 
)
from utils_notebook.testing import validate_season_series
from utils_notebook.constants import ADDR_BEANSTALK
from utils_notebook.queries import QueryManager
from utils_notebook.css import css_tooltip_timeseries_multi_colored

In [2]:
sg, bs = load_subgraph()
q = QueryManager(sg, bs) 

In [3]:
@cache
def query_field_daily_snapshots(**kwargs): 
    return q.query_field_daily_snapshots(fields=['season', 'timestamp', 'weather']) 

In [4]:
df_field = query_field_daily_snapshots(cache=1).copy()
df_field = df_field.rename(columns={'weather': 'temperature'})
df_field.temperature = df_field.temperature / 100
df_field.tail()

,season,timestamp,temperature
340,8145,2022-10-31 23:00:11,40.58
341,8169,2022-11-01 23:00:11,40.54
342,8193,2022-11-02 23:00:11,40.30
343,8217,2022-11-03 23:00:11,39.92
344,8220,2022-11-04 02:00:11,39.83


In [5]:
id_vars = ['timestamp', 'season']
value_vars = ['temperature']
df = wide_to_longwide(df_field, "timestamp", id_vars, value_vars, sort_col="timestamp") 
df.head()

,timestamp,season,variable,value,temperature
0,2021-08-07 00:06:08,25,temperature,0.39,0.39
1,2021-08-08 00:08:15,49,temperature,0.35,0.35
2,2021-08-09 00:15:57,73,temperature,0.39,0.39
3,2021-08-10 00:01:29,97,temperature,0.11,0.11
4,2021-08-11 00:00:05,121,temperature,0.01,0.01


In [9]:
colors = {'temperature': Batlow_5.hex_colors[0]}
format = '.0%'
c = (
    chart(
        df, 
        "timestamp", 
        lmetrics=['temperature'], 
        lstrategy="line", 
        tooltip_formats={'temperature': format},
        yaxis_left_kwargs=dict(title="Temperature", format=format),
        hide_legend=True, 
        colors=colors, 
        show_exploit_rule=True, 
        exploit_day=16, 
    )
)

css_lines = css_tooltip_timeseries_multi_colored(value_vars, colors) 
css = "\n".join(css_lines)

apply_css("")
# apply_css(css)

c

/Users/ALEX/opt/anaconda3/envs/beanstalk/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.LayerChart(...)

In [7]:
# TODO: update css for this chart 
output_chart(c, css=css)

<IPython.core.display.JSON object>